# Model 3

## Pakete und Daten laden

In einem ersten Schritt laden wir die nöten Pakete und laden die Daten.

In [1]:
import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer
import os
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance
from pathlib import Path

target = Path("C:/Users/mhu/Documents/github/topic_model_it")
os.chdir(target)

data_model_3 = pd.read_csv("data/informatikkurse.csv")
data_model_3 = data_model_3[data_model_3['model_3'] == 1].copy()
data_model_3

c:\Users\mhu\miniconda3\envs\bertopic-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\mhu\AppData\Local\Temp\ipykernel_19768\3492676078.py:25: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data_model_3 = pd.read_csv("data/informatikkurse.csv")


,Unnamed: 0,veranstaltung_id,veranstaltung_titel,kursbeschreibung,kursformat_original,sprache_original,lernziele,lernmethode,literatur,voraussetzungen,...,medizin_info,wirt_info,recht_info,geo_info,bau_info,base_info,model_1_b,model_2,model_3,model_4
2,3,1717915,Übung zu: Ethik in der Informatik,NaN,Übung,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,1,1,1,0
3,4,1717989,Übung zu: Ethik in der Informatik,NaN,Übung,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,1,1,1,0
6,7,1720482,Exercise: Automatic Graph Drawing,NaN,Übung,Englisch,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,1,1,1,0
8,9,1717878,Übung zu: Einführung in die Informatik,NaN,Übung,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,1,1,1,0
19,20,1717925,Inf-FD-LeLeG: Lehren und Lernen im Fach Inform...,NaN,Vorlesung,NaN,NaN,NaN,NaN,Anmeldung über die StudiDB . Achtung: Bitte me...,...,0,0,0,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51243,51244,1710590,Exercise: Big Data Management and Analytics,NaN,Übung,Englisch,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,1,1,1,0
51244,51245,1710591,inf-DSProj-01a: Data Science Projekt,Moodle,Praktikum,NaN,NaN,NaN,NaN,Gewünschte aber nicht verpflichtend für die Te...,...,0,0,0,0,0,1,1,1,1,0
51245,51246,1710592,infADMMLM-01a: Advanced Data Mining and Machin...,NaN,Vorlesung,Englisch,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,1,1,1,0
51246,51247,1710593,infBDMaA-01a: Big Data Management and Analytics,NaN,Vorlesung,Englisch,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,1,1,1,0


## Reproduzierbarkeit durch Seeds
Im folgenden fixieren wir die Zufallszahlen-Generatoren aller beteiligten Bibliotheken auf den Wert 11. Dies stellt sicher, dass die Experimente bei jedem Durchlauf identische Ergebnisse liefern.

In [2]:
seed = 11  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

## Docs

Wir generieren als nächsten die Dokumente, die das Eingangsmaterial für das Topic-Model bilden.

In [3]:
# Anzeige der Spaltennamen von data_model_3
print(data_model_3.columns.tolist())

data_model_3 = data_model_3[["veranstaltung_titel", "kursbeschreibung"]].copy()
data_model_3.head()

data_model_3 = data_model_3.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
data_model_3['titel_kursbesch'] = data_model_3['veranstaltung_titel'] + ' ' + data_model_3['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = data_model_3['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

['Unnamed: 0', 'veranstaltung_id', 'veranstaltung_titel', 'kursbeschreibung', 'kursformat_original', 'sprache_original', 'lernziele', 'lernmethode', 'literatur', 'voraussetzungen', 'zusatzinformationen', 'anmerkungen', 'pruefung', 'dozierende', 'teilnehmerzahl', 'sws', 'ects', 'url', 'nummer', 'pfad', 'scrape_datum', 'hochschule_id', 'hochschule_name', 'fakultaet_id', 'fakultaet_name', 'organisation_id', 'organisation_name', 'semester_id', 'semester_name', 'sprache_id', 'sprache_name', 'kursformat_id', 'kursformat_name', 'matchingart_id', 'matchingart_name', 'lehr_und_forschungsbereich_id', 'bio_info', 'ingen_info', 'medien_info', 'medizin_info', 'wirt_info', 'recht_info', 'geo_info', 'bau_info', 'base_info', 'model_1_b', 'model_2', 'model_3', 'model_4']


## Stopwords

In [4]:
from utils import stopwords_config

irrelevant_terms = stopwords_config.irrelevant_terms

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms)
irrelevant_terms

['vl',
 'übung',
 'übungen',
 'seminar',
 'arbeitsgruppenseminar',
 'oberseminar',
 'proseminar',
 'blockveranstaltung',
 'vorlesung',
 'kolloquium',
 'theoriekolloquium',
 'einführung',
 'tutorium',
 'ue',
 'vereinbarung',
 'projekt',
 'praktikum',
 'masterprojekt',
 'wiederholerklausur',
 'fortgeschrittenenpraktikum',
 'hauptseminar',
 'fachpraktikum',
 'ergänzungsvorlesung',
 'forschungspraktikum',
 'begleitseminar',
 'abschlussarbeiten',
 'unterrichtspraktikum',
 'masterseminar',
 'proseminare',
 'praxisseminar',
 'praxissemester',
 'schulpraxis',
 'ringpraktikum',
 'basispraktikum',
 'praxistage',
 'industriepraktikum',
 'vorkurs',
 'projektseminar',
 'juniorprofessur',
 'masterarbeiten',
 'forschungsseminar',
 'modulbeschreibung',
 'veranstaltung',
 'kommentare',
 'raum',
 'uhrzeit',
 'vereinbarung',
 'vorlesung',
 'praktikum',
 'masterprojekt',
 'wiederholerklausur',
 'fortgeschrittenenpraktikum',
 'hauptseminar',
 'fachpraktikum',
 'ergänzungsvorlesung',
 'forschungspraktikum',

# Model

Wir definieren unser Model.

In [5]:
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("Current device:", model.device)

CUDA available: True
Current device: cuda:0


## Model-Settings, GPU-Check
Konfiguration frei wählbar (einfach im Code unten anpassen).
Hier kann eine erste explorative Untersuchung durchgeführt werden.

Wir checken weiterhin, ob wirklich unsere GPU verwendet wird. Wenn `cuda:0` ist dies der Fall.

In [6]:
# CountVectorizer
vectorizer = CountVectorizer(
  stop_words=sw,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
  token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
  ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
)

# Embedding Settings  
embedding_model = SentenceTransformer(
    "paraphrase-multilingual-mpnet-base-v2",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# UMAP Settings
umap_model = UMAP(
    n_neighbors=10,
    n_components=10,
    min_dist=0.0,
    metric="cosine",
    random_state=seed
)
# HDBSCAN Settings
hdbscan_model = HDBSCAN(
    min_cluster_size=100,
    cluster_selection_epsilon=0.2,
    prediction_data=True
)
# Representation Settings
representation_model = MaximalMarginalRelevance(diversity=0.1)

# BERTopic initialisieren
data_model_3 = BERTopic(
  embedding_model=embedding_model,
  # min_topic_size=100,
  nr_topics="auto", 
  language="multilingual",
  verbose=True,
  umap_model=umap_model,
  vectorizer_model=vectorizer,
  hdbscan_model=hdbscan_model,
  top_n_words = 20,
  representation_model=representation_model
)

## Training

Wir trainieren das Modell.

In [7]:
data_model_3.fit(docs)
data_model_3.get_topic_info()

2026-02-05 15:37:37,558 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 193/193 [00:04<00:00, 43.32it/s] 
2026-02-05 15:37:42,119 - BERTopic - Embedding - Completed ✓
2026-02-05 15:37:42,120 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-02-05 15:37:56,520 - BERTopic - Dimensionality - Completed ✓
2026-02-05 15:37:56,520 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-02-05 15:37:56,749 - BERTopic - Cluster - Completed ✓
2026-02-05 15:37:56,749 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2026-02-05 15:37:57,448 - BERTopic - Representation - Completed ✓
2026-02-05 15:37:57,451 - BERTopic - Topic reduction - Reducing number of topics
2026-02-05 15:37:57,455 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-02-05 15:37:58,491 - BERTopic - Representation - Completed ✓
2026-02-05 15:37:58,495 - BERTopic - Topic reduction - R

,Topic,Count,Name,Representation,Representative_Docs
0,-1,958,-1_data_2 01_systems_informatik,"[data, 2 01, systems, informatik, algorithms, ...",[ \t Dodo Alive! - Resurrecting the Dodo with ...
1,0,3080,0_software_informatik_systems_systeme,"[software, informatik, systems, systeme, engin...",[Inf-MPSE: Master Project - Software Engineeri...
2,1,715,1_learning_machine learning_deep learning_3d,"[learning, machine learning, deep learning, 3d...",[Machine Learning and Artificial Intelligence ...
3,2,517,2_algorithmen_process mining_algorithms_datens...,"[algorithmen, process mining, algorithms, date...",[ Advanced Practical Course - There and Back A...
4,3,265,3_quantum_cryptography_quantum computing_krypt...,"[quantum, cryptography, quantum computing, kry...",[Seminar: 2.01.814-A Advances in Security & Pr...
5,4,257,4_bioinformatik_biometrie_bioinformatics_epide...,"[bioinformatik, biometrie, bioinformatics, epi...","[Bioinformatik Forschungspraktikum , Grundlage..."
6,5,147,5_data_medizinische_medizinische informatik_he...,"[data, medizinische, medizinische informatik, ...",[Semantic Interoperability in Health - Data ma...
7,6,123,6_control_robotics_robotik_robot,"[control, robotics, robotik, robot, robots, ro...",[ Remote Machine Intelligence Lab ●\tOverview...
8,7,107,7_datenbanken informationssysteme_datenbanksys...,"[datenbanken informationssysteme, datenbanksys...",[Übung: Datenbanken IIA: Datenbank-Entwurf (Üb...


## Topic Visualiszieren, Outlier Reduzieren, Topics mergen

In [8]:
data_model_3.visualize_topics()
#data_model_3.visualize_hierarchy() # Hilft extrem zu sehen, welche Themen zusammengehören

In [ ]:
# topic_model.merge_topics(docs, [0, 13])

# # Outlier reduzieren
# # BERTopic auf Test-Daten anwenden
# topics, probs = data_model_3.transform(docs)
# print(data_model_3.get_topic_freq())

# # # Outlier reduzieren
# topics = data_model_3.reduce_outliers(docs, topics)
# data_model_3.update_topics(docs, topics=topics)

In [ ]:
data_model_3.get_topic_info()

## Labeling

Mit Hilfe von gpt4o lassen wir uns die Topcis labeln. Das erleichtert die Interpretation des Modells

In [9]:
from openai import OpenAI as OpenAIClient
from bertopic.representation import OpenAI as OpenAIRep

client = OpenAIClient()  # nutzt OPENAI_API_KEY aus der Umgebung
rep = OpenAIRep(
    client=client, 
    model="gpt-4o-mini", 
    delay_in_seconds=10,      # Erhöhe Delay
    exponential_backoff=True, # Aktiviere das schrittweise längere Warten
    nr_docs=3                 # Weniger Dokumente pro Topic reduzieren die Token-Last
)
data_model_3.update_topics(docs, representation_model=rep)
data_model_3.get_topic_info() 

100%|██████████| 9/9 [01:36<00:00, 10.73s/it]


,Topic,Count,Name,Representation,Representative_Docs
0,-1,958,-1_Robotic Dodo Project,[Robotic Dodo Project],[ \t Dodo Alive! - Resurrecting the Dodo with ...
1,0,3080,0_Software Development Project,[Software Development Project],[Inf-MPSE: Master Project - Software Engineeri...
2,1,715,1_Artificial Intelligence Education,[Artificial Intelligence Education],[Machine Learning and Artificial Intelligence ...
3,2,517,2_Process Mining Education,[Process Mining Education],[ Advanced Practical Course - There and Back A...
4,3,265,3_Advances in Security,[Advances in Security],[Seminar: 2.01.814-A Advances in Security & Pr...
5,4,257,4_Bioinformatics and Evolution,[Bioinformatics and Evolution],"[Bioinformatik Forschungspraktikum , Grundlage..."
6,5,147,5_Medical Informatics Applications,[Medical Informatics Applications],[Semantic Interoperability in Health - Data ma...
7,6,123,6_Robotics and Teleoperation,[Robotics and Teleoperation],[ Remote Machine Intelligence Lab ●\tOverview...
8,7,107,7_Database Design Principles,[Database Design Principles],[Übung: Datenbanken IIA: Datenbank-Entwurf (Üb...


In [ ]:
# Speichern
topic_model.save("models/explor_model_1", serialization="safetensors", save_embedding_model=True)

# Laden
from bertopic import BERTopic
loaded_model = BERTopic.load("models/explor_model_1")